In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append(os.path.dirname(sys.path[0]))

In [2]:
import pandas as pd
from databases.postgresql import PostgresDB, Session

In [44]:
dates = [
'2023/12/28',
'2023/12/29',
'2023/12/30',
'2023/12/31',
'2024/01/01',
'2024/01/02']

dates_pairs = list()
for i in range(0, len(dates) - 1):
    dates_pairs.append([dates[i], dates[i+1]])
dates_pairs

[['2023/12/28', '2023/12/29'],
 ['2023/12/29', '2023/12/30'],
 ['2023/12/30', '2023/12/31'],
 ['2023/12/31', '2024/01/01'],
 ['2024/01/01', '2024/01/02']]

In [82]:
schemas = ['chain_0x38', 'chain_0x1', 'chain_0xfa', 'chain_0x89',
            'chain_0xa', 'chain_0xa4b1', 'chain_0xa86a', 'chain_0x2b6653dc']

tables_sizes = dict()

tables = ['amount_in_out',	'dapp_interaction',	'smart_contract',	'token_decimals',	'token_transfer', 'wrapped_token']

for schema in schemas:
    tables_sizes[schema] = pd.read_csv(f'../data/{schema}_tables_size.csv', index_col='time')
    tables_sizes[schema] = tables_sizes[schema].loc[dates, tables]
    tables_sizes[schema] = tables_sizes[schema].apply(pd.to_numeric)



In [83]:
def subtract_rows(df: pd.DataFrame, date_1: str, date_2):
    _diffs = (df.loc[date_2] - df.loc[date_1])
    _diffs = _diffs.to_frame().T
    _diffs['time'] = date_2
    _diffs.set_index('time', inplace=True)
    return _diffs

subtract_rows(df=tables_sizes[schemas[3]], date_2='2024/01/02', date_1='2024/01/01')

,amount_in_out,dapp_interaction,smart_contract,token_decimals,token_transfer,wrapped_token
time,,,,,,
2024/01/02,9764864,29351936,0,57344,1443471360,236642304


In [86]:
differences = dict()
for schema in schemas:
    # differences[schema] = pd.DataFrame()
    for _i, _dates_pair in enumerate(dates_pairs):
        _diffs = subtract_rows(df=tables_sizes[schema], date_2=_dates_pair[1], date_1=_dates_pair[0])
        if _i < 1:
            differences[schema] = _diffs
        else:
            differences[schema] = pd.concat([differences[schema], _diffs])
            
    differences[schema]['sum'] = differences[schema].sum(axis=1)

In [87]:
print(schemas[3])
differences[schemas[3]]

chain_0x89


,amount_in_out,dapp_interaction,smart_contract,token_decimals,token_transfer,wrapped_token,sum
time,,,,,,,
2023/12/29,-2351104,32038912,0,65536,78692352,713064448,821510144
2023/12/30,6709248,44318720,8192,40960,141484032,876765184,1069326336
2023/12/31,2785280,53207040,0,65536,1295654912,1004724224,2356436992
2024/01/01,-245760,59015168,0,65536,145793024,700121088,904749056
2024/01/02,9764864,29351936,0,57344,1443471360,236642304,1719287808


In [91]:
total_increases = dict()
for _date in dates[1:]:
    _sum_all = 0
    for schema in schemas:
        _sum_all += differences[schema].loc[_date, 'sum']
    total_increases[_date] = _sum_all
    
total_increases

{'2023/12/29': 2794250240.0,
 '2023/12/30': 3818176512.0,
 '2023/12/31': 4276551680.0,
 '2024/01/01': 5561319424.0,
 '2024/01/02': 4297875456.0}

In [92]:
differences['chain_0x38']

,amount_in_out,dapp_interaction,smart_contract,token_decimals,token_transfer,wrapped_token,sum
time,,,,,,,
2023/12/29,40280064,171638784,0,204800,285523968,39149568,536797184
2023/12/30,89194496,139264,0,172032,1143644160,36511744,1269661696
2023/12/31,-40263680,428204032,0,163840,58204160,26714112,473022464
2024/01/01,-83853312,192020480,0,155648,2575785984,56164352,2740273152
2024/01/02,-12976128,199270400,0,155648,6324224,39067648,231841792
